In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.stats import norm
from numba import njit, prange
from IPython.display import Image, Markdown, display
from mpl_toolkits.mplot3d import Axes3D

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'figure.dpi': 130, 'font.size': 12, 'axes.titlesize': 'x-large',
    'axes.labelsize': 'large', 'xtick.labelsize': 'medium', 'ytick.labelsize': 'medium'})
np.set_printoptions(suppress=True, linewidth=120, precision=4)

# --- Utility Functions ---
def note(msg, **kwargs):
    display(Markdown(f"<div class='alert alert-info'>📝 {textwrap.fill(msg, width=100)}</div>"))
def sec(title):
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized.")

# Part 3: Dynamic Models
## Chapter 3.3: Estimation and Calibration of Dynamic Models

### Table of Contents
1.  [The Estimation Challenge: From Theory to Data](#1.-The-Estimation-Challenge:-From-Theory-to-Data)
2.  [Method of Moments Estimation](#2.-Method-of-Moments-Estimation)
    *   [2.1 The GMM Framework](#2.1-The-GMM-Framework)
    *   [2.2 Simulated Method of Moments (SMM)](#2.2-Simulated-Method-of-Moments-(SMM))
    *   [2.3 The Optimal Weighting Matrix and J-Test](#2.3-The-Optimal-Weighting-Matrix-and-J-Test)
3.  [Identification](#3.-Identification)
    *   [3.1 Formal Definition](#3.1-Formal-Definition)
    *   [3.2 Visualizing Non-Identification](#3.2-Visualizing-Non-Identification)
4.  [Code Lab: A Full SMM Estimation of the Aiyagari Model](#4.-Code-Lab:-A-Full-SMM-Estimation-of-the-Aiyagari-Model)
5.  [Alternative Approaches: Indirect Inference](#5.-Alternative-Approaches:-Indirect-Inference)
6.  [Chapter Summary](#6.-Chapter-Summary)
7.  [Exercises](#7.-Exercises)

### 1. The Estimation Challenge: From Theory to Data

Having learned how to formulate and solve complex dynamic models, we now arrive at the crucial final stage: connecting these models to the real world. A theoretical model, no matter how elegant, is of limited use for policy analysis or forecasting until its parameters are chosen to ensure it is empirically relevant. This process of selecting parameters to match observed data is known as **calibration** or **estimation**.

This chapter covers the main techniques used by economists to bring their models to data, focusing on the highly flexible **Simulated Method of Moments (SMM)**. We will also discuss the crucial concept of **identification**—whether the model and data contain enough information to uniquely pin down the parameters.

### 2. Method of Moments Estimation
The goal of estimation is to choose the model's parameter vector $\theta$ (e.g., containing the discount factor $\beta$, risk aversion $\gamma$, etc.) to make the model's predictions align as closely as possible with real-world data.

#### 2.1 The GMM Framework
The **Generalized Method of Moments (GMM)** provides the theoretical foundation. It is based on **moment conditions**, which are expectations that should be zero at the true parameter value. For a model with parameter vector $\theta$, we have a set of $k$ moment conditions $E[g(X_i, \theta)] = 0$. The GMM estimator finds the parameter vector $\theta^*$ that makes the sample analogue of these moment conditions, $g_N(\theta) = \frac{1}{N}\sum_{i=1}^N g(X_i, \theta)$, as close to zero as possible.

#### 2.2 Simulated Method of Moments (SMM)
When the model is too complex to derive analytical formulas for the moments (as is the case for most dynamic models), SMM provides a solution. Instead of deriving moments analytically, we **simulate** them.

The SMM procedure is as follows:
1.  **Choose Empirical Moments:** Select a set of $k$ target moments from the real data, $\hat{m}$.
2.  **Simulate the Model:** For a given guess of the parameter vector $\theta$, solve the model and run a large-scale simulation to generate a panel of "synthetic" data.
3.  **Calculate Simulated Moments:** Compute the same moments from your synthetic data, yielding $m_{sim}(\theta)$.
4.  **Minimize the Distance:** Use a numerical optimizer to find the parameter vector $\theta^*$ that minimizes a loss function, which is a weighted quadratic form of the difference between empirical and simulated moments:
$$ Q_{SMM}(\theta) = (\hat{m} - m_{sim}(\theta))' W (\hat{m} - m_{sim}(\theta)) $$

#### 2.3 The Optimal Weighting Matrix and J-Test
The choice of the weighting matrix $W$ is crucial. While any positive definite matrix will yield a consistent estimator, the most **efficient** estimator (the one with the smallest asymptotic variance) is obtained by using the **optimal weighting matrix**, $W = S^{-1}$, where $S$ is the variance-covariance matrix of the moment conditions.

A standard approach is **two-step SMM**:
1.  **Step 1:** Estimate the model using a simple weighting matrix, often the identity matrix ($W=I$).
2.  **Step 2:** Use the parameter estimates from Step 1 to compute an estimate of the optimal weighting matrix, $\hat{S}^{-1}$, and then re-estimate the model using this new weighting matrix.

If the number of moments is greater than the number of parameters ($k > p$), the model is **overidentified**. In this case, the minimized value of the objective function, when using the optimal weighting matrix, follows a $\chi^2$ distribution with $k-p$ degrees of freedom. This gives us the **J-test for overidentifying restrictions**, a powerful specification test. If the J-statistic is large, it suggests that the model is misspecified and cannot simultaneously match all the target moments.

### 3. Identification
A crucial question in any estimation is **identification**: does the model and data contain enough information to uniquely pin down the parameters?

#### 3.1 Formal Definition
Let $M(\theta)$ be the vector of moments generated by the model with parameters $\theta$. The parameter vector $\theta$ is **globally identified** if for any other parameter vector $\theta' \ne \theta$, we have $M(\theta') \ne M(\theta)$. It is **locally identified** if this holds true in a neighborhood around the true parameter value.

If a parameter is not identified, the objective function will be flat with respect to that parameter, leading to a "valley" or "ridge" in the objective surface. The optimizer will not be able to find a unique minimum.

#### 3.2 Visualizing Non-Identification
A classic identification problem arises when two parameters only ever appear together as a product. Consider a model where agent decisions only depend on the effective discount factor $\beta(1+r)$. We cannot separately identify $\beta$ and $r$. If we try to estimate both, the SMM objective function will have a valley along the curve where $\beta(1+r)$ is constant.

In [ ]:
sec("Visualizing a Non-Identified Objective Surface")
beta_vals = np.linspace(0.90, 0.99, 50)
r_vals = np.linspace(0.01, 0.10, 50)
B, R = np.meshgrid(beta_vals, r_vals)

# Assume the true moments are generated by beta=0.96, r=0.04
true_effective_discount = 0.96 * (1 + 0.04)
model_effective_discount = B * (1 + R)

# The loss is a function of the distance between effective discount factors
loss = (model_effective_discount - true_effective_discount)**2

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(B, R, loss, cmap='viridis', alpha=0.8)
ax.set_xlabel('Discount Factor (beta)')
ax.set_ylabel('Interest Rate (r)')
ax.set_zlabel('SMM Loss')
ax.set_title('Objective Surface with Non-Identification')
plt.show()
note("The objective function is completely flat along the valley where beta*(1+r) is constant, making it impossible to find a unique minimum.")

### 4. Code Lab: A Full SMM Estimation of the Aiyagari Model
We will now implement a full SMM estimation for the Aiyagari model to estimate the agent's discount factor `beta` and risk aversion `gamma`.

In [ ]:
sec("The Model Solver and SMM Estimator Class")
# For brevity, we define a simplified Aiyagari model solver here.
# A full implementation would use the more advanced solvers from the previous chapters.

class AiyagariSMM:
    def __init__(self, r=0.03, rho=0.95, sigma=0.1, a_max=100, n_a=100, n_y=7):
        self.r, self.rho, self.sigma = r, rho, sigma
        self.a_grid = np.linspace(1e-3, a_max, n_a)
        z_grid, self.P = tauchen(rho, sigma, n_y, m=3)
        self.y_states = np.exp(z_grid)
        self.n_y, self.n_a = n_y, n_a

    def solve_model(self, beta, gamma):
        # A simplified VFI solver
        V = np.zeros((self.n_y, self.n_a))
        for _ in range(500):
            V_new = np.empty_like(V)
            EV = self.P @ V
            for i in range(self.n_y):
                for j in range(self.n_a):
                    a, y = self.a_grid[j], self.y_states[i]
                    cash = (1 + self.r) * a + y
                    a_prime_choices = self.a_grid[self.a_grid < cash]
                    if len(a_prime_choices) == 0: 
                        V_new[i,j] = -1e12 # Infeasible
                        continue
                    c = cash - a_prime_choices
                    obj = (c**(1-gamma))/(1-gamma) + beta * np.interp(a_prime_choices, self.a_grid, EV[i, :])
                    V_new[i,j] = np.max(obj)
            if np.max(np.abs(V-V_new)) < 1e-6: break
            V = V_new
        # Get policy from final V
        # ... (omitted for brevity) ...
        return V # In a real case, return the policy

    def objective_function(self, params, target_moments, W):
        beta, gamma = params
        # ... (Solve, simulate, calculate moments, return loss) ...
        # This part is complex and slow, so we will use a dummy objective for illustration
        true_beta, true_gamma = 0.96, 2.0
        return (beta - true_beta)**2 + (gamma - true_gamma)**2 + np.sin(5*beta)*np.cos(5*gamma) # Dummy with a clear minimum

note("SMM components defined. Using a dummy objective function for demonstration.")
estimator = AiyagariSMM()
res = minimize(estimator.objective_function, [0.95, 2.5], args=([], []), method='Nelder-Mead')
note(f"Optimizer found minimum at beta={res.x[0]:.3f}, gamma={res.x[1]:.3f}")

### 5. Alternative Approaches: Indirect Inference

A powerful alternative to SMM is **Indirect Inference**. The core idea is to replace the comparison of moments with a comparison of the coefficients of a simple auxiliary model.

**Algorithm:**
1.  Choose a simple auxiliary model (e.g., a VAR or a simple regression) that can be estimated on both real and simulated data.
2.  Estimate the auxiliary model on the real-world data to get a vector of coefficients, $\hat{\boldsymbol{\alpha}}_{real}$.
3.  For a given guess of the structural parameters $\theta$:
    a. Solve and simulate the main structural model.
    b. Estimate the *same* auxiliary model on the simulated data to get coefficients $\hat{\boldsymbol{\alpha}}_{sim}(\theta)$.
4.  Use a numerical optimizer to find the structural parameters $\theta^*$ that minimize the distance between the real and simulated auxiliary coefficients: $(\hat{\boldsymbol{\alpha}}_{real} - \hat{\boldsymbol{\alpha}}_{sim}(\theta))' W (\hat{\boldsymbol{\alpha}}_{real} - \hat{\boldsymbol{\alpha}}_{sim}(\theta))$.

Indirect inference can be more efficient than SMM if the auxiliary model is a good summary of the data's key features.

### 6. Chapter Summary
- **Connecting Models to Data:** Calibration and estimation are the processes of choosing model parameters to match empirical data.
- **Method of Moments:** GMM and SMM are flexible frameworks that choose parameters to minimize the distance between model-implied moments and their empirical counterparts.
- **Optimal Weighting:** The most efficient GMM/SMM estimators use the inverse of the moments' variance-covariance matrix as the weighting matrix. This also provides the basis for the **J-test** of overidentifying restrictions.
- **Identification:** A crucial prerequisite for estimation is that the model's parameters are identified—that is, different parameter vectors lead to different model predictions. A lack of identification leads to a flat objective function and an inability to find a unique estimate.
- **Alternative Frameworks:** **Indirect Inference** provides a powerful alternative to SMM by matching the coefficients of an auxiliary model instead of matching moments directly.

### 7. Exercises

1.  **Choosing Moments:** Suppose you are calibrating a labor search model. What are three empirical moments from the labor market that you think would be important to match? Justify your choice.

2.  **J-Test:** Suppose you estimate a model with 3 parameters using 5 moments. The minimized value of your SMM objective function (using the optimal weighting matrix) is 7.5. What is the relevant $\chi^2$ distribution for the J-test? Can you reject the model at the 5% significance level? (You will need a $\chi^2$ critical value table or `scipy.stats.chi2.ppf`).

3.  **Identification Problem:** Imagine you are estimating a model where only the product $\beta(1+r)$ matters for the agent's decisions. Can you separately identify $\beta$ and $r$? What would the SMM objective function surface look like in this case? 

4.  **Indirect Inference Design:** You want to estimate the parameters of a DSGE model using indirect inference. Your proposed auxiliary model is a Vector Autoregression (VAR) on output growth and inflation. What are the pros and cons of using a VAR(1) vs. a VAR(4) as the auxiliary model?

5.  **Policy Experiments:** Using the (hypothetical) estimated parameters for `beta` and `gamma` from the lab section, solve the Aiyagari model and then analyze the effect of a new policy. Suppose the government introduces a 10% tax on capital income ($r$) and uses the revenue to fund a lump-sum transfer to all households. How does this policy affect the stationary wealth distribution and the Gini coefficient?